# Previsão de renda

### 4 elementos importantes
- Esse notebook
- Streamlit com as análises
- Seu Github com o projeto
- Vídeo no readme do github mostrando o streamlit

## Etapa 1 CRISP - DM: Entendimento do negócio

<span style="color:red">Objetivo de negócio: prever a renda mensal de uma pessoa com base em características socioeconômicas.

Aplicação prática: pode ser usado em concessão de crédito, segmentação de clientes, análise de risco.

Questões de negócio:

Quais fatores mais influenciam a renda? (educação, tempo de emprego, idade, tipo de renda).

É possível prever a renda de forma confiável para novos clientes? </span>


## Etapa 2 Crisp-DM: Entendimento dos dados
<span style="color:red">Na segunda etapa é o entendimento dos dados. Foram fornecidas 14 variaveis mais a variavel resposta para verificação. </span>


### Dicionário de dados

<span style="color:red">Os dados estão dispostos em uma tabela com uma linha para cada cliente, e uma coluna para cada variável armazenando as características desses clientes. Colocamos uma cópia o dicionário de dados (explicação dessas variáveis) abaixo neste notebook:</span>


| Variável                | Descrição                                           | Tipo         |
| ----------------------- | --------------------------------------------------- | ------------ |
| data_ref                | Data de referência da informação do cliente         | Data         |
| id_cliente              | Identificador único do cliente                      | Numérico / Chave |
| sexo                    | Sexo do cliente (ex.: M, F)                         | Categórico   |
| posse_de_veiculo        | Indica se o cliente possui veículo                  | Binário (Sim/Não) |
| posse_de_imovel         | Indica se o cliente possui imóvel                   | Binário (Sim/Não) |
| qtd_filhos              | Quantidade de filhos do cliente                     | Numérico     |
| tipo_renda              | Fonte principal de renda (ex.: assalariado, autônomo)| Categórico  |
| educacao                | Nível de escolaridade do cliente                    | Categórico   |
| estado_civil            | Estado civil do cliente (solteiro, casado etc.)     | Categórico   |
| tipo_residencia         | Tipo de residência (alugada, própria, financiada)   | Categórico   |
| idade                   | Idade do cliente (em anos)                          | Numérico     |
| tempo_emprego           | Tempo de emprego do cliente (em anos)               | Numérico     |
| qt_pessoas_residencia   | Quantidade de pessoas que moram na residência       | Numérico     |
| renda                   | Renda mensal do cliente                             | Numérico     |




#### Carregando os pacotes
É considerado uma boa prática carregar os pacotes que serão utilizados como a primeira coisa do programa.

<span style="color:red">ESCREVER AQUI</span>


In [ ]:
!pip install ydata_profiling

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from ydata_profiling import ProfileReport
from pandas_profiling import ProfileReport
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [ ]:
files.upload()

#### Carregando os dados
O comando pd.read_csv é um comando da biblioteca pandas (pd.) e carrega os dados do arquivo csv indicado para um objeto *dataframe* do pandas.

<span style="color:red">ESCREVER AQUI</span>


In [ ]:
renda = pd.read_csv('previsao_de_renda.csv')

In [ ]:
renda

In [ ]:
renda.drop(['data_ref', 'Unnamed: 0', 'index'], axis=1, inplace=True)

In [ ]:
renda1 = renda.copy()

In [ ]:
renda.head()

#### Entendimento dos dados - Univariada
Nesta etapa tipicamente avaliamos a distribuição de todas as variáveis.

In [ ]:
prof = ProfileReport(renda, explorative=True, minimal=True)
prof


In [ ]:
prof.to_file('renda_analisys.html')

MOSTRA A ESTATISTICA DE CADA COLUNA DO DATAFRAME. NO INCIO PODE VERIFICAR O "ALERT", QUE MOSTRA COLUNAS QUE PODEM APRESENTAR PROBLEMAS QUANDO FORMOS FAZER A ANALISE


### Entendimento dos dados - Bivariadas




<span style="color:red">O GRAFICO MOSTRA QUE MAIS JOVENS TENDEM A SER MAUS PAGADORES, QUE TAMBEM PODE SER INDICADO POR ALGUMAS INFLUENCIAS EXTERNAS.</span>


In [ ]:
print(renda1['mau'].value_counts())
print("\nTaxa de inadimplentes:")
print(renda1['mau'].mean())

In [ ]:
renda["mau"] = renda["mau"].astype(int)

renda["faixa_idade"] = pd.qcut(renda["idade"], 4)

biv = renda.groupby("faixa_idade")["mau"].mean()

biv.plot(kind="bar")
plt.ylabel("Proporção de inadimplentes")
plt.xlabel("Faixa de idade")
plt.title("Inadimplência por faixa de idade")
plt.show()

## Etapa 3 Crisp-DM: Preparação dos dados
Nessa etapa realizamos tipicamente as seguintes operações com os dados:

 - **seleção**: Já temos os dados selecionados adequadamente?
 - **limpeza**: Precisaremos identificar e tratar dados faltantes
 - **construção**: construção de novas variáveis
 - **integração**: Temos apenas uma fonte de dados, não é necessário integração
 - **formatação**: Os dados já se encontram em formatos úteis?



<span style="color:red">ESCREVER AQUI</span>


In [ ]:
metadata = pd.DataFrame(renda.dtypes, columns = ['tipo'])

metadata['n_categorias'] = 0

for var in metadata.index:
    metadata.loc[var,'n_categorias'] = len(renda.groupby([var]).size())

metadata

In [ ]:
qtd_total = renda.shape[0]
qtd_adiplentes = renda[renda['mau'] == False].shape[0]
qtd_inadiplentes = renda[renda['mau'] == True].shape[0]

print(f'A proprorção de adiplentes é de {round(100*qtd_adiplentes / qtd_total, 2)}%')
print(f'A proprorção de inadiplentes é de {round(100 * qtd_inadiplentes / qtd_total, 2)}%')

In [ ]:
renda1.select_dtypes('object').describe().transpose()

In [ ]:
renda1.select_dtypes('number').describe().transpose()

In [ ]:
renda1.info()

In [ ]:
def convert_dummy(renda1, feature,rank=0):
    pos = pd.get_dummies(renda1[feature], prefix=feature)
    mode = renda1[feature].value_counts().index[rank]
    biggest = feature + '_' + str(mode)
    pos.drop([biggest],axis=1,inplace=True)
    renda1.drop([feature],axis=1,inplace=True)
    renda1=renda1.join(pos)
    return renda1

In [ ]:
for var in metadata[metadata['tipo'] == 'object'].index:
    renda1 = convert_dummy(renda1, var)

In [ ]:
renda1['tempo_emprego'] = renda1['tempo_emprego'].fillna(renda1['tempo_emprego'].mean())
renda1.head()

In [ ]:
renda1.dtypes, renda1.info()

## Etapa 4 Crisp-DM: Modelagem
Nessa etapa que realizaremos a construção do modelo. Os passos típicos são:
- Selecionar a técnica de modelagem
- Desenho do teste
- Avaliação do modelo


<span style="color:red">ESCREVER AQUI</span>


### Rodando o modelo


<span style="color:red">ESCREVER AQUI</span>


In [ ]:
X = renda1.drop('mau', axis=1)
y = renda['mau']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

## Etapa 5 Crisp-DM: Avaliação dos resultados


<span style="color:red">ESCREVER AQUI</span>


## Etapa 6 Crisp-DM: Implantação
Nessa etapa colocamos em uso o modelo desenvolvido, normalmente implementando o modelo desenvolvido em um motor que toma as decisões com algum nível de automação.